## IAMSI -- 2022-2023

# TME 07: Règles d'association


<font color="RED" size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms**


_Double-cliquer ici et insérer les noms et prénoms de votre binôme_


## Présentation


### But de ce TME

Ce TME a pour but de réaliser une implémentation _intuitive_ de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Contenu de l'archive téléchargée

Une fois détarée, l'archive **tme-07.tgz** crée le répertoire `tme07` contenant :

- un répertoire `dataset` contenant des bases d'exemples pour tester les algorithmes
- deux fichiers exécutables (Linux): `apriori32` et `fpgrowth32` (ces 2 programmes ont été développés par Christian Borgelt, plus d'infos sur sa page web https://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance. Si vous n'êtes pas sous Linux, vous pouvez trouver une version pour votre OS ici:
  - `apriori` : https://borgelt.net/apriori.html
  - `fpgrowth` : https://borgelt.net/fpgrowth.html


#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce notebook complété par les réponses aux questions posées.

Ce compte-rendu est à poster :

- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un complément peut être envoyé **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : _Nom1_Nom2.ipynb_

- _Nom1_ et _Nom2_ : noms des membres du binôme
- ne pas compresser ou faire une archive : envoyer le notebook tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les joindre au message.


In [1]:
# Vérification de la version de Python utilisée:
import sys

sys.path  # le path doit contenir python3.5 ou une version supérieure


['/home/aymeric/Documents/DAC/S2/IAMSI/tme07',
 '/home/aymeric/.pyenv/versions/3.10.10/lib/python310.zip',
 '/home/aymeric/.pyenv/versions/3.10.10/lib/python3.10',
 '/home/aymeric/.pyenv/versions/3.10.10/lib/python3.10/lib-dynload',
 '',
 '/home/aymeric/.pyenv/versions/3.10.10/lib/python3.10/site-packages']

In [2]:
from pprint import pprint


## Traitement d'une base d'apprentissage

### Chargement de la base

On commence par travailler sur la base exemple du fichier `exemple-1.txt` (fourni dans le répertoire datasets).

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :


In [3]:
import csv

with open("datasets/exemple-1.txt", "r") as fichier:
    lecteur = csv.reader(fichier, delimiter=" ")
    i = 0
    for ligne in lecteur:
        i += 1
        print("ligne", i, ":", ligne, set(ligne))


ligne 1 : ['a', 'b', 'c'] {'c', 'a', 'b'}
ligne 2 : ['a', 'd', 'e'] {'d', 'e', 'a'}
ligne 3 : ['b', 'c', 'd'] {'c', 'd', 'b'}
ligne 4 : ['a', 'b', 'c', 'd'] {'c', 'd', 'a', 'b'}
ligne 5 : ['b', 'c'] {'c', 'b'}
ligne 6 : ['a', 'b', 'd'] {'d', 'a', 'b'}
ligne 7 : ['d', 'e'] {'d', 'e'}
ligne 8 : ['a', 'b', 'c', 'd'] {'c', 'd', 'a', 'b'}
ligne 9 : ['c', 'd', 'e'] {'c', 'd', 'e'}
ligne 10 : ['a', 'b', 'c'] {'c', 'a', 'b'}


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `chargeBase` qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des `set()` donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble Python plutôt que comme une liste. Il est ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.


In [4]:
def chargeBase(filepath, delimiteur=" "):
    """Renvoie les itemsets contenus dans un fichier texte.

    Parameters
    ----------
    filename : str
        Chemin vers le fichier.
    delimiteur : str, default=" "
        Le caractère utilisé pour séparer les données.

    Returns
    -------
    base : dict of set
        Ensemble des transactions.
        Clé : numéro de la transaction (numéro de la ligne).
        Valeur : itemset correspondant.
    """
    import csv

    base = {}
    with open(filepath, "r") as fichier:
        lecteur = csv.reader(fichier, delimiter=delimiteur)
        for row, ligne in enumerate(lecteur):
            base[row] = set(ligne)
    return base


In [5]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase("datasets/exemple-1.txt")
Base1


Résultat du chargement de 'exemple-1.txt', on obtient : 


{0: {'a', 'b', 'c'},
 1: {'a', 'd', 'e'},
 2: {'b', 'c', 'd'},
 3: {'a', 'b', 'c', 'd'},
 4: {'b', 'c'},
 5: {'a', 'b', 'd'},
 6: {'d', 'e'},
 7: {'a', 'b', 'c', 'd'},
 8: {'c', 'd', 'e'},
 9: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.


## Itemsets et support


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `noms_items` qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.


In [6]:
def noms_items(base):
    """Renvoie l'ensemble des items composant une base.

    Parameters
    ----------
    base : dict of set
        Ensemble des transactions.

    Returns
    -------
    set
        Items uniques de la base.
    """
    return set().union(*base.values())


In [7]:
print("Pour la BASE précédente :")
noms_items(Base1)


Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `singletons` qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

_Remarque_ : attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [8]:
def singletons(base):
    """Rend la liste des itemsets de taille 1 d'une base.

    Parameters
    ----------
    base : dict of set
        Ensemble des transactions.

    Returns
    -------
    list of set
        Liste des singletons de la base.
    """
    return [{singleton} for singleton in noms_items(base)]


In [9]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)


Exemple: singletons(Base1) rend :


[{'b'}, {'d'}, {'e'}, {'c'}, {'a'}]

<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `comptage` qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.


In [10]:
def comptage(base, itemset):
    """Renvoie le nombre de transactions contenant un itemset dans une base.

    Parameters
    ----------
    base : dict of set
        Ensemble des transactionss.
    itemset : set
        Itemset.

    Returns
    -------
    int
        Nombre de transactions contenant l'itemset donné.
    """
    return sum([itemset.issubset(transaction) for transaction in base.values()])


In [11]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1, {'a','b','c'}) rend la valeur %s." % comptage(Base1, {"a", "b", "c"}))
print("comptage(Base1, {'a'}) rend la valeur %s." % comptage(Base1, {"a"}))


Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1, {'a','b','c'}) rend la valeur 4.
comptage(Base1, {'a'}) rend la valeur 6.


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `support` qui, pour une BASE et un itemset donné, rend le support de cet itemset dans la BASE.


In [12]:
comptage(Base1, {"a", "b", "c"})


def support(base, itemset):
    """Calcule le support d'un itemset d'une base.

    .. math:: \text{support} = \frac {n_{\text{itemset}}} {n}

    Parameters
    ----------
    base : dict of set
        Ensemble des itemsets.
    itemset : set
        Itemset.

    Returns
    -------
    float
        Support de l'itemset donné.
    """
    return comptage(base, itemset) / len(base)


In [13]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : " + str(support(Base1, {"a", "b", "c"})))
print("Support de l'itemset {'a'} dans la base précédente : " + str(support(Base1, {"a"})))


Support de l'itemset {'a','b','c'} dans la base précédente : 0.4
Support de l'itemset {'a'} dans la base précédente : 0.6


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire _datasets_ (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `apriori_gen` qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.

*Rappel de l'algorithme* :

<font style="font-variant: small-caps">Apriori-Gen</font>($F$)<br>
Entrée : $F$ ensemble d’itemsets fréquents de longueur $k$<br>
$C \leftarrow \{c = f_1 ∪ f_2 \in F \times F$ tels que $|c| = k + 1\}$<br>
pour chaque $c \in C$ faire<br>
&emsp; pour chaque $i \in c$ faire<br>
&emsp;&emsp;si $c \setminus \{i\} \in F$ alors<br>
&emsp;&emsp;&emsp;$C \leftarrow C \setminus \{c\}$<br>
retourner $C$<br>

In [14]:
def apriori_gen(itemsets):
    """Variante avec deux boucles."""
    C = []
    k = len(itemsets[0])
    for f1 in itemsets:
        for f2 in itemsets:
            c = f1.union(f2)
            if len(c) == k + 1 and c not in C:
                C.append(c)
    for c in C[:]:
        for i in c:
            if c.difference({i}) not in itemsets and c in C:
                C.remove(c)
    return C
    
%timeit apriori_gen([{"a"}, {"b"}, {"c"}, {"d"}])
%timeit apriori_gen([{"a", "b"}, {"a", "d"}, {"b", "d"}, {"b", "c"}, {"c", "d"}])

3.89 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
5.35 µs ± 191 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [15]:
from itertools import combinations

def apriori_gen(itemsets):
    """Applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats
    à partir d'une liste d'itemsets.

    Parameters
    ----------
    itemsets : list
        Liste d'itemsets de même longueur k.

    Returns
    -------
    list
        Liste des itemsets candidats de longueurs k+1.
    """
    C = []
    k = len(itemsets[0])
    for f1, f2 in combinations(itemsets, 2):
        c = f1.union(f2)
        if len(c) == k + 1 and c not in C:
            C.append(c)
    for c in C[:]:
        for i in c:
            if c.difference({i}) not in itemsets and c in C:
                C.remove(c)
    return C
    
%timeit apriori_gen([{"a"}, {"b"}, {"c"}, {"d"}])
%timeit apriori_gen([{"a", "b"}, {"a", "d"}, {"b", "d"}, {"b", "c"}, {"c", "d"}])

2.69 µs ± 55.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.66 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [16]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{"a"}, {"b"}, {"c"}, {"d"}]))
print("-------------------------------------------------------------------------------")
print("Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend ")
print(apriori_gen([{"a", "b"}, {"a", "d"}, {"b", "d"}, {"b", "c"}, {"c", "d"}]))


Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'a', 'b'}, {'c', 'a'}, {'d', 'a'}, {'c', 'b'}, {'d', 'b'}, {'c', 'd'}]
-------------------------------------------------------------------------------
Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'d', 'a', 'b'}, {'c', 'd', 'b'}]


<font color="RED" size="+1">**[Q]**</font> Ecrire la fonction `apriori` qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.


In [17]:
def apriori(base, minsup):
    """Renvoie la liste des itemsets trouvés par apriori.

    Parameters
    ----------
    base : dict of set
        Ensemble des transactions.
    minsup : float
        Seuil minimal pour le support.

    Returns
    -------
    list of tuple of (set, float)
        (itemset, support)
    """
    C = [singletons(base)]
    F = []
    k = 0
    while len(C[k]) > 0:
        Fk = []
        for c in C[k]:
            s = support(base, c)
            if s >= minsup:
                Fk.append(c)
                F.append((c, s))
        gen = apriori_gen(Fk)
        C.append(gen)
        k += 1
    return F


%timeit apriori(Base1, 0.3)


28.5 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
print("Exemple: apriori(Base1, 0.3) rend ")
pprint(apriori(Base1, 0.3))


Exemple: apriori(Base1, 0.3) rend 
[({'b'}, 0.7),
 ({'d'}, 0.7),
 ({'e'}, 0.3),
 ({'c'}, 0.7),
 ({'a'}, 0.6),
 ({'d', 'b'}, 0.4),
 ({'c', 'b'}, 0.6),
 ({'a', 'b'}, 0.5),
 ({'d', 'e'}, 0.3),
 ({'c', 'd'}, 0.4),
 ({'d', 'a'}, 0.4),
 ({'c', 'a'}, 0.4),
 ({'c', 'd', 'b'}, 0.3),
 ({'d', 'a', 'b'}, 0.3),
 ({'c', 'a', 'b'}, 0.4)]


## Expérimentations


### Vérification avec la base du TD

Utiliser la fonction `apriori` avec la base de transactions du TD afin de vérifier que les itemsets obtenus sont corrects.


In [19]:
BaseTD = chargeBase("datasets/td.txt")
pprint(BaseTD)
print("Singletons :", singletons(BaseTD))


{0: {'d', 'e', 'a', 'b'},
 1: {'c', 'd', 'b'},
 2: {'d', 'e', 'a', 'b'},
 3: {'c', 'd', 'e', 'a'},
 4: {'c', 'd', 'e', 'b'},
 5: {'d', 'e', 'b'},
 6: {'c', 'd'},
 7: {'c', 'a', 'b'},
 8: {'d', 'e', 'a'},
 9: {'d', 'b'}}
Singletons : [{'b'}, {'d'}, {'e'}, {'c'}, {'a'}]


In [20]:
print("Exemple : apriori(BaseTD, 0.3) rend :")
pprint(apriori(BaseTD, 0.3))


Exemple : apriori(BaseTD, 0.3) rend :
[({'b'}, 0.7),
 ({'d'}, 0.9),
 ({'e'}, 0.6),
 ({'c'}, 0.5),
 ({'a'}, 0.5),
 ({'d', 'b'}, 0.6),
 ({'e', 'b'}, 0.4),
 ({'c', 'b'}, 0.3),
 ({'a', 'b'}, 0.3),
 ({'d', 'e'}, 0.6),
 ({'c', 'd'}, 0.4),
 ({'d', 'a'}, 0.4),
 ({'e', 'a'}, 0.4),
 ({'d', 'e', 'b'}, 0.4),
 ({'d', 'e', 'a'}, 0.4)]


### Comparaisons avec les implémentations apriori et fpgrowth


<font color="RED" size="+1">**[Q]**</font> Tester votre fonction `apriori` sur les 3 bases données dans datasets et confronter les résultats obtenus avec ceux fournis par l'exécutable `apriori`.


In [21]:
print("Résultat du chargement de 'mushrooms.txt', on obtient : ")
Base_mushrooms = chargeBase("datasets/mushrooms.txt", delimiteur=",")
# Base_mushrooms
print(len(Base_mushrooms), " transactions.")


Résultat du chargement de 'mushrooms.txt', on obtient : 
8416  transactions.


In [22]:
print("Pour la BASE précédente :")
print(noms_items(Base_mushrooms))
print("Exemple: singletons(Base_mushrooms) rend :")
print(singletons(Base_mushrooms))
print("Nombre de singletons :", len(singletons(Base_mushrooms)))


Pour la BASE précédente :
{'GRASSES', 'ONE', 'KNOBBED', 'FIBROUS', 'SCATTERED', 'FOUL', 'BULBOUS', 'CLOSE', 'SUNKEN', 'PINK', 'NO', 'FREE', 'CREOSOTE', 'BLACK', 'URBAN', 'CINNAMON', '?', 'BELL', 'NUMEROUS', 'FISHY', 'ANISE', 'NONE', 'SILKY', 'TAPERING', 'MUSTY', 'CHOCOLATE', 'GRAY', 'SMOOTH', 'GREEN', 'PARTIAL', 'ATTACHED', 'TWO', 'WOODS', 'FLAT', 'FLARING', 'YELLOW', 'CLUB', 'BRUISES', 'PUNGENT', 'PATHS', 'EVANESCENT', 'POISONOUS', 'SPICY', 'SCALY', 'CLUSTERED', 'BUFF', 'SEVERAL', 'PURPLE', 'ALMOND', 'CROWDED', 'GROOVES', 'MEADOWS', 'LEAVES', 'SOLITARY', 'EDIBLE', 'ENLARGING', 'RED', 'NARROW', 'PENDANT', 'BROWN', 'ABUNDANT', 'WASTE', 'CONVEX', 'BROAD', 'ROOTED', 'LARGE', 'CONICAL', 'ORANGE', 'EQUAL', 'WHITE'}
Exemple: singletons(Base_mushrooms) rend :
[{'GRASSES'}, {'ONE'}, {'KNOBBED'}, {'FIBROUS'}, {'SCATTERED'}, {'FOUL'}, {'BULBOUS'}, {'CLOSE'}, {'SUNKEN'}, {'PINK'}, {'NO'}, {'FREE'}, {'CREOSOTE'}, {'BLACK'}, {'URBAN'}, {'CINNAMON'}, {'?'}, {'BELL'}, {'NUMEROUS'}, {'FISHY'}, {'ANISE

In [23]:
pprint(apriori(Base_mushrooms, 0.9))


[({'ONE'}, 0.9230038022813688),
 ({'FREE'}, 0.9743346007604563),
 ({'PARTIAL'}, 1.0),
 ({'WHITE'}, 0.9771863117870723),
 ({'ONE', 'FREE'}, 0.9001901140684411),
 ({'ONE', 'PARTIAL'}, 0.9230038022813688),
 ({'ONE', 'WHITE'}, 0.9001901140684411),
 ({'FREE', 'PARTIAL'}, 0.9743346007604563),
 ({'FREE', 'WHITE'}, 0.9743346007604563),
 ({'PARTIAL', 'WHITE'}, 0.9771863117870723),
 ({'ONE', 'FREE', 'PARTIAL'}, 0.9001901140684411),
 ({'ONE', 'FREE', 'WHITE'}, 0.9001901140684411),
 ({'ONE', 'PARTIAL', 'WHITE'}, 0.9001901140684411),
 ({'FREE', 'PARTIAL', 'WHITE'}, 0.9743346007604563),
 ({'PARTIAL', 'ONE', 'FREE', 'WHITE'}, 0.9001901140684411)]


In [24]:
print("Résultat du chargement de 'titanic-red.csv', on obtient : ")
Base_titanic = chargeBase("datasets/titanic-red.csv", delimiteur=",")
print(len(Base_titanic), " transactions.")


Résultat du chargement de 'titanic-red.csv', on obtient : 
741  transactions.


In [25]:
print("Pour la BASE Titanic :")
noms_items(Base_titanic)
print("Exemple: singletons(Base_titanic) rend :")
singletons(Base_titanic)


Pour la BASE Titanic :
Exemple: singletons(Base_titanic) rend :


[{'GuernseyMontclairNJand/orToledoOhio'},
 {'AuburnNY'},
 {'BrynMawrPAUSA'},
 {'NewForestEngland'},
 {'HaverfordPACooperstownNY'},
 {'BronxNY'},
 {'KrakuddenSwedenMouneIL'},
 {'ProvoUT'},
 {'AberdeenPortlandOR'},
 {'NewBritainCT'},
 {'BromsgroveEnglandMontrealPQ'},
 {'RotterdamNetherlands'},
 {'HaddenfieldNJ'},
 {'StroodKentEnglandDetroitMI'},
 {'EnglandSanFranciscoCA'},
 {'MexicoCityMexico'},
 {'AscotBerkshireRochesterNY'},
 {'BirkdaleEnglandClevelandOhio'},
 {'LondonMiddlesex'},
 {'BridgeruleDevon'},
 {'SyriaYoungstownOH'},
 {'LondonStatenIslandNY'},
 {'MerrillWI'},
 {'GuernseyEnglandEdgewoodRI'},
 {'HelsinkiFinlandAshtabulaOhio'},
 {'SeattleWA'},
 {'EnglandOglesbyIL'},
 {'MontrealPQ'},
 {'BelgiumMontrealPQ'},
 {'ForesvikNorwayPortlandND'},
 {'VancouverBC'},
 {'IllinoisUSA'},
 {'Greece'},
 {'RotherfieldSussexEnglandEssexCoMA'},
 {'GermantownPhiladelphiaPA'},
 {'SanFranciscoCA'},
 {'ParisMontrealPQ'},
 {'StJamesLongIslandNY'},
 {'BrooklineMA'},
 {'JanjgirIndiaPennsylvania'},
 {'Vadsbr

In [26]:
Lres = apriori(Base_titanic, 0.2)

# tri sur les valeurs de supports:
# print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
# print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1], reverse=True):
    if len(t[0]) > 1:
        print(t)


({'dead', 'male'}, 0.46963562753036436)
({'Southampton', 'male'}, 0.46288798920377866)
({'dead', 'Southampton'}, 0.41295546558704455)
({'dead', 'Southampton', 'male'}, 0.3684210526315789)
({'alive', 'female'}, 0.32388663967611336)
({'Southampton', '2nd'}, 0.30634278002699056)
({'Southampton', 'alive'}, 0.2982456140350877)
({'Southampton', 'female'}, 0.2483130904183536)
({'1st', 'alive'}, 0.23076923076923078)
({'1st', 'male'}, 0.21997300944669365)
({'Southampton', '1st'}, 0.20917678812415655)
({'2nd', 'male'}, 0.20647773279352227)
({'Southampton', 'alive', 'female'}, 0.203778677462888)


<font color="RED" size="+1">**[Q]**</font> Utiliser l'éxécutable `apriori` pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance).


In [27]:
%%capture
!./apriori -trs30 datasets/exemple-1.txt out/exemple-1-trs30.out
!./apriori -trc20 datasets/exemple-1.txt out/exemple-1-trc20.out
!./apriori -trs90 datasets/mushrooms.txt out/mushrooms-trs90.out
!./apriori -trs90c95 datasets/mushrooms.txt out/mushrooms-trs90c95.out
!./apriori -trs20m2n2 datasets/titanic-red.csv out/titanic-red-trs20m2n2.out
!./apriori -trc99 datasets/titanic-red.csv out/titanic-red-trc99.out

In [28]:
from IPython.display import Pretty
Pretty(data="out/exemple-1-trs30.out")

d <- e (30, 100)
b <- a c (40, 100)
c <- a b (50, 80)
b <- a (60, 83.3333)
b <- c (70, 85.7143)
c <- b (70, 85.7143)


In [29]:
Pretty(data="out/exemple-1-trc20.out")

d <- e a (10, 100)
a <- e d (30, 33.3333)
e <- a d (40, 25)
a <- e (30, 33.3333)
c <- e d (30, 33.3333)
d <- e c (10, 100)
e <- d c (40, 25)
d <- e (30, 100)
e <- d (70, 42.8571)
c <- e (30, 33.3333)
e <-  (100, 30)
b <- a d c (20, 100)
c <- a d b (30, 66.6667)
d <- a c b (40, 50)
a <- d c b (30, 66.6667)
c <- a d (40, 50)
d <- a c (40, 50)
a <- d c (40, 50)
b <- a d (40, 75)
d <- a b (50, 60)
a <- d b (40, 75)
d <- a (60, 66.6667)
a <- d (70, 57.1429)
b <- a c (40, 100)
c <- a b (50, 80)
a <- c b (60, 66.6667)
c <- a (60, 66.6667)
a <- c (70, 57.1429)
b <- a (60, 83.3333)
a <- b (70, 71.4286)
a <-  (100, 60)
b <- d c (40, 75)
c <- d b (40, 75)
d <- c b (60, 50)
c <- d (70, 57.1429)
d <- c (70, 57.1429)
b <- d (70, 57.1429)
d <- b (70, 57.1429)
d <-  (100, 70)
b <- c (70, 85.7143)
c <- b (70, 85.7143)
c <-  (100, 70)
b <-  (100, 70)


In [30]:
Pretty(data="out/mushrooms-trs90.out")

CLOSE <- ONE FREE WHITE PARTIAL (90.019, 82.7878)
CLOSE <- ONE FREE WHITE (90.019, 82.7878)
CLOSE <- ONE FREE PARTIAL (90.019, 82.7878)
CLOSE <- ONE FREE (90.019, 82.7878)
CLOSE <- ONE WHITE PARTIAL (90.019, 82.7878)
CLOSE <- ONE WHITE (90.019, 82.7878)
CLOSE <- ONE PARTIAL (92.3004, 83.2132)
CLOSE <- ONE (92.3004, 83.2132)
CLOSE <- FREE WHITE PARTIAL (97.4335, 80.5854)
CLOSE <- FREE WHITE (97.4335, 80.5854)
CLOSE <- FREE PARTIAL (97.4335, 80.5854)
CLOSE <- FREE (97.4335, 80.5854)
CLOSE <- WHITE PARTIAL (97.7186, 80.642)
CLOSE <- WHITE (97.7186, 80.642)
CLOSE <- PARTIAL (100, 81.0837)
CLOSE <-  (100, 81.0837)
PARTIAL <- ONE FREE WHITE (90.019, 100)
WHITE <- ONE FREE PARTIAL (90.019, 100)
FREE <- ONE WHITE PARTIAL (90.019, 100)
ONE <- FREE WHITE PARTIAL (97.4335, 92.3902)
WHITE <- ONE FREE (90.019, 100)
FREE <- ONE WHITE (90.019, 100)
ONE <- FREE WHITE (97.4335, 92.3902)
PARTIAL <- ONE FREE (90.019, 100)
FREE <- ONE PARTIAL (92.3004, 97.5283)
ONE <- FREE PARTIAL (97.4335, 92.3902)
FREE 

In [31]:
Pretty(data="out/mushrooms-trs90c95.out")

PARTIAL <- ONE FREE WHITE (90.019, 100)
WHITE <- ONE FREE PARTIAL (90.019, 100)
FREE <- ONE WHITE PARTIAL (90.019, 100)
WHITE <- ONE FREE (90.019, 100)
FREE <- ONE WHITE (90.019, 100)
PARTIAL <- ONE FREE (90.019, 100)
FREE <- ONE PARTIAL (92.3004, 97.5283)
FREE <- ONE (92.3004, 97.5283)
PARTIAL <- ONE WHITE (90.019, 100)
WHITE <- ONE PARTIAL (92.3004, 97.5283)
WHITE <- ONE (92.3004, 97.5283)
PARTIAL <- ONE (92.3004, 100)
PARTIAL <- FREE WHITE (97.4335, 100)
WHITE <- FREE PARTIAL (97.4335, 100)
FREE <- WHITE PARTIAL (97.7186, 99.7082)
WHITE <- FREE (97.4335, 100)
FREE <- WHITE (97.7186, 99.7082)
PARTIAL <- FREE (97.4335, 100)
FREE <- PARTIAL (100, 97.4335)
FREE <-  (100, 97.4335)
PARTIAL <- WHITE (97.7186, 100)
WHITE <- PARTIAL (100, 97.7186)
WHITE <-  (100, 97.7186)
PARTIAL <-  (100, 100)


In [32]:
Pretty(data="out/titanic-red-trs20m2n2.out")

Southampton <- 2nd (34.8178, 87.9845)
alive <- female (38.8664, 83.3333)
male <- dead (53.4413, 87.8788)


In [33]:
Pretty(data="out/titanic-red-trc99.out")

<font color="RED" size="+1">**[Q]**</font> Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


In [34]:
%%capture
!./apriori -tr -el datasets/td.txt out/td-lift.out
!./apriori -tr -ez datasets/td.txt out/td-RR.out

In [35]:
Pretty(data="out/td-lift.out")

d <- c a e (10, 100)
e <- c a d (10, 100)
d <- c e b (10, 100)
d <- c e (20, 100)
d <- c (50, 80)
d <- a e b (20, 100)
e <- a b d (20, 100)
d <- a e (40, 100)
e <- a d (40, 100)
e <- a (50, 80)
d <- a (50, 80)
d <- e b (40, 100)
d <- e (60, 100)
d <- b (70, 85.7143)
d <-  (100, 90)


In [36]:
Pretty(data="out/td-RR.out")

d <- c a e (10, 100)
e <- c a d (10, 100)
d <- c e b (10, 100)
d <- c e (20, 100)
d <- a e b (20, 100)
e <- a b d (20, 100)
d <- a e (40, 100)
e <- a d (40, 100)
e <- a (50, 80)
d <- e b (40, 100)
d <- e (60, 100)


<font color="RED" size="+1">**[Q]**</font> Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.

Par exemple :

\begin{align}
\text{Interest}(X \longrightarrow Y) & = \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
\text{IS}(X \longrightarrow Y) & = \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}


### Utilisation de fp-growth


<font color="RED" size="+1">**[Q]**</font> Utiliser `fpgrowth` pour générer des règles d'association sur les différentes
bases de données. Comparer avec les résultats obtenus dans la section précédente. En particulier, comparer les temps d'éxecution.


In [37]:
%%capture
!./fpgrowth -trs30 datasets/exemple-1.txt out/exemple-1-trs30.out
!./fpgrowth -trs90 datasets/mushrooms.txt out/mushrooms-trs90.out
!./fpgrowth -trs20m2n2 datasets/titanic-red.csv out/titanic-red-trs20m2n2.out

In [38]:
Pretty(data="out/exemple-1-trs30.out")

c <- b (70, 85.7143)
b <- c (70, 85.7143)
c <- a b (50, 80)
b <- a c (40, 100)
b <- a (60, 83.3333)
d <- e (30, 100)


In [39]:
Pretty(data="out/mushrooms-trs90.out")

PARTIAL <-  (100, 100)
PARTIAL <- WHITE (97.7186, 100)
WHITE <- PARTIAL (100, 97.7186)
WHITE <-  (100, 97.7186)
PARTIAL <- FREE (97.4335, 100)
FREE <- PARTIAL (100, 97.4335)
PARTIAL <- FREE WHITE (97.4335, 100)
WHITE <- FREE PARTIAL (97.4335, 100)
FREE <- WHITE PARTIAL (97.7186, 99.7082)
WHITE <- FREE (97.4335, 100)
FREE <- WHITE (97.7186, 99.7082)
FREE <-  (100, 97.4335)
PARTIAL <- ONE (92.3004, 100)
ONE <- PARTIAL (100, 92.3004)
PARTIAL <- ONE WHITE (90.019, 100)
WHITE <- ONE PARTIAL (92.3004, 97.5283)
ONE <- WHITE PARTIAL (97.7186, 92.1206)
WHITE <- ONE (92.3004, 97.5283)
ONE <- WHITE (97.7186, 92.1206)
PARTIAL <- ONE FREE (90.019, 100)
FREE <- ONE PARTIAL (92.3004, 97.5283)
ONE <- FREE PARTIAL (97.4335, 92.3902)
PARTIAL <- ONE FREE WHITE (90.019, 100)
WHITE <- ONE FREE PARTIAL (90.019, 100)
FREE <- ONE WHITE PARTIAL (90.019, 100)
ONE <- FREE WHITE PARTIAL (97.4335, 92.3902)
WHITE <- ONE FREE (90.019, 100)
FREE <- ONE WHITE (90.019, 100)
ONE <- FREE WHITE (97.4335, 92.3902)
FREE <- 

In [40]:
Pretty(data="out//titanic-red-trs20m2n2.out")

male <- dead (53.4413, 87.8788)
alive <- female (38.8664, 83.3333)
Southampton <- 2nd (34.8178, 87.9845)


In [41]:
%timeit !!./apriori -trs30 datasets/exemple-1.txt out/exemple-1-trs30.out
%timeit !!./apriori -trs90 datasets/mushrooms.txt out/mushrooms-trs90.out
%timeit !!./apriori -trs20m2n2 datasets/titanic-red.csv out/titanic-red-trs20m2n2.out

1.72 ms ± 80.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
8.3 ms ± 246 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.12 ms ± 60.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [42]:
%timeit !!./fpgrowth -trs30 datasets/exemple-1.txt out/exemple-1-trs30.out
%timeit !!./fpgrowth -trs90 datasets/mushrooms.txt out/mushrooms-trs90.out
%timeit !!./fpgrowth -trs20m2n2 datasets/titanic-red.csv out/titanic-red-trs20m2n2.out

1.7 ms ± 34.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
7.81 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.03 ms ± 83.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Petit t-test pour comparer parce que c'est pas très flagrant... :thinking:

<font color="RED" size="+1">**[Q]**</font> En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.


In [43]:
%%capture
!!./fpgrowth -trs90n2c95 -ez datasets/mushrooms.txt out/mushrooms-trs90n2c95.out
!!./fpgrowth -trs20m3n3c80 -el datasets/titanic-red.csv out/titanic-red-trs20m3n3c80.out

In [44]:
Pretty(data="out/mushrooms-trs90n2c95.out")

PARTIAL <-  (100, 100)
PARTIAL <- WHITE (97.7186, 100)
PARTIAL <- FREE (97.4335, 100)
WHITE <- FREE (97.4335, 100)
FREE <- WHITE (97.7186, 99.7082)
PARTIAL <- ONE (92.3004, 100)


In [45]:
Pretty(data="out/titanic-red-trs20m3n3c80.out")

male <- dead Southampton (41.2955, 89.2157)
alive <- female Southampton (24.8313, 82.0652)
Southampton <- 2nd male (20.6478, 88.2353)
dead <- 2nd male (20.6478, 83.6601)


## Annexes : utilisation des exécutables _apriori_ et _fpgrowth_

La documentation complète de ces programmes est disponible aux URL suivantes :

- pour _apriori_ : https://www.borgelt.net/doc/apriori/apriori.html
- pour _fp-growth_ : https://www.borgelt.net/doc/fpgrowth/fpgrowth.html

Leur format d'utilisation général est :

        ./<programme> [options] infile [outfile]

(où <`programme`> est soit _apriori_, soit _fpgrowth_).

Les options de base de ces 2 programmes sont :

- sans argument : génération des itemsets fréquents (argument `-ts` activé par défaut)
- _tr_ : pour obtenir les règles d'associations
- _s_ : pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- _m_ : pour fournir un nombre minimum d'items dans un itemset
- _n_ : pour fournir un nombre maximum d'items dans un itemset

Par exemple (commandes lancées dans le répertoire père du répertoire _datasets/_) :

        ./apriori -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out
        ./fpgrowth -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out
